In [ ]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

In [ ]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import geopandas as gpd
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

from kapipy.gis import GISK

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('STATS_API_KEY')

output_folder = r"c:\temp\data"

#layers
meshblock2022_centroid_layer_id = "106716"  #Meshblock 2022 (Centroid True) - 56,966 points
territorial_authority2025_layer_id = "120963" #Territorial Authority 2025 - 68 polygons

#tables
current_geographic_boundaries_table_id = "114916" # Current Geographic Boundaries Table - 44 Rows

## Server object  
Get a reference to the LINZ server  

In [ ]:
statsnz = GISK(name="statsnz", api_key=api_key)
print(statsnz)

## Layer item  
Run some tests on a layer item  

In [ ]:
itm = statsnz.content.get(meshblock2022_centroid_layer_id)
print(itm)

In [ ]:
print(f'{itm.supports_changesets=}')
print(f'{itm.data.fields=}')
print(f'{itm.data.crs=}')
print(f'{itm.data.geometry_type=}')

In [ ]:
waikato_polygon = gpd.read_file('../tests/spatial_data/waikato.json')
print(waikato_polygon)

# read shapefile into a GeoDataFrame
matamata_gdf = gpd.read_file("../tests/spatial_data/matamata_piako.shp")
matamata_gdf.head()

In [ ]:
#Query using WFS
res = itm.query(count=100, out_sr=2193, bbox=matamata_gdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
job = itm.export("geodatabase", out_sr=2193, extent=matamata_gdf,)

In [ ]:
print(job)

In [ ]:
dl = job.download(folder=output_folder)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Table item  
Run some tests on a table item

In [ ]:
table_itm = statsnz.content.get(current_geographic_boundaries_table_id)
print(table_itm)
print(f'{table_itm.supports_changesets=}')

In [ ]:
#Query using WFS to get DataFrame
data = table_itm.query(count=100)
logging.info(f"Total records returned {table_itm.title}: {data.shape[0]}")
data.head()

In [ ]:
#Query using WFS to get raw JSON
data = table_itm.query_json(count=10)
logging.info(f"Total records returned {table_itm.title}: {len(data['features'])}")
print(data)

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job)

In [ ]:
output_folder = r"c:\temp\data"
dl = job.download(folder=output_folder)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = statsnz.content.get(meshblock2022_centroid_layer_id)
itm2 = statsnz.content.get(territorial_authority2025_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export("geodatabase", out_sr=2193, extent=matamata_gdf,)
job2 = itm2.export("geodatabase", out_sr=2193, extent=matamata_gdf,)

output_folder = r"c:\temp\data\multiple"
results = statsnz.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)
